This notebook is used to assign postcodes to four datasets for use in the subsequent modeling phase. The first dataset contains historical rental prices, retrieved from the Canvas announcements. The other three datasets include SA2 area information, along with income and population data, sourced from the Australian Bureau of Statistics (ABS).

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import requests

## Past Rental Price

Retrieved from the Applied Data Science (MAST30034) Canvas announcement on September 3.

In [2]:
# Read file
past_rent = pd.read_excel('../data/raw/internal/Moving annual rent by suburb - March quarter 2023.xlsx', sheet_name='All properties')
past_rent = past_rent.iloc[:,1:]
past_rent.head()

,Unnamed: 1,Lease commenced in year ending,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 178,Unnamed: 179,Unnamed: 180,Unnamed: 181,Unnamed: 182,Unnamed: 183,Unnamed: 184,Unnamed: 185,Unnamed: 186,Unnamed: 187
0,NaN,Mar 2000,NaN,Jun 2000,NaN,Sep 2000,NaN,Dec 2000,NaN,Mar 2001,...,Mar 2022,NaN,Jun 2022,NaN,Sep 2022,NaN,Dec 2022,NaN,Mar 2023,NaN
1,NaN,Count,Median,Count,Median,Count,Median,Count,Median,Count,...,Count,Median,Count,Median,Count,Median,Count,Median,Count,Median
2,Albert Park-Middle Park-West St Kilda,1143,260,1134,260,1177,270,1178,275,1208,...,867,500,855,515,881,500,832,525,786,545
3,Armadale,733,200,737,200,738,205,739,210,718,...,805,430,851,450,852,450,840,460,751,490
4,Carlton North,864,260,814,260,799,265,736,270,718,...,581,580,535,595,547,600,546,600,490,620


Convert merge first two rows as new column name.

In [3]:
df = past_rent

# First row contains the year/quarter information
columns_first_row = df.iloc[0, :].tolist()  

# Has some nan values in first row, assign it to its previous value
new_columns_first_row  = [] 
# Index of nan values
nan_indices = [i for i, val in enumerate(columns_first_row) if isinstance(val, float) and np.isnan(val)]

for i in range(len(columns_first_row)):
    if i in nan_indices and i !=0:
        new_columns_first_row.append(columns_first_row[i-1])
    else:
        new_columns_first_row.append(columns_first_row[i])
print(new_columns_first_row[:5])


# Second row contains 'Count'/'Median'
columns_second_row = df.iloc[1, :].tolist()  # Second row as a list
print(columns_second_row[:5])

[nan, 'Mar 2000', 'Mar 2000', 'Jun 2000', 'Jun 2000']
[nan, 'Count', 'Median', 'Count', 'Median']


In [4]:
# Create new column names by combining the two rows
new_column_names = []
for col1, col2 in zip(new_columns_first_row, columns_second_row):
    if pd.isna(col1):
        new_column_names.append(col2)  
    else:
        new_column_names.append(f"{col1}-{col2}")

# Drop the first two rows (since they are now used as column names)
df = df.drop([0, 1])

# Assign the new column names
df.columns = new_column_names
df.columns = ['suburb' if pd.isna(col) else col for col in df.columns] # Name first column 
df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce') # Change data types

df.head()

,suburb,Mar 2000-Count,Mar 2000-Median,Jun 2000-Count,Jun 2000-Median,Sep 2000-Count,Sep 2000-Median,Dec 2000-Count,Dec 2000-Median,Mar 2001-Count,...,Mar 2022-Count,Mar 2022-Median,Jun 2022-Count,Jun 2022-Median,Sep 2022-Count,Sep 2022-Median,Dec 2022-Count,Dec 2022-Median,Mar 2023-Count,Mar 2023-Median
2,Albert Park-Middle Park-West St Kilda,1143.0,260.0,1134,260,1177.0,270.0,1178.0,275.0,1208.0,...,867,500,855,515,881,500,832,525,786,545
3,Armadale,733.0,200.0,737,200,738.0,205.0,739.0,210.0,718.0,...,805,430,851,450,852,450,840,460,751,490
4,Carlton North,864.0,260.0,814,260,799.0,265.0,736.0,270.0,718.0,...,581,580,535,595,547,600,546,600,490,620
5,Carlton-Parkville,1339.0,260.0,1304,260,1300.0,260.0,1320.0,260.0,1273.0,...,6143,310,6018,319,6871,340,6627,350,6690,400
6,CBD-St Kilda Rd,2132.0,320.0,2264,320,2358.0,320.0,2361.0,320.0,2591.0,...,17845,365,16792,390,18284,419,17627,450,17426,500


In [5]:
def calculate_yearly_values(df, years):
    ''' 
        This function calculate yearly aggregated values from quarterly data for a given dataframe.
    '''
    for year in years:
        # Define the count and median columns for each quarter
        count_columns = [f'Mar {year}-Count', f'Jun {year}-Count', f'Sep {year}-Count', f'Dec {year}-Count']
        median_columns = [f'Mar {year}-Median', f'Jun {year}-Median', f'Sep {year}-Median', f'Dec {year}-Median']
        
        # Filter out columns that are not present in the dataframe
        valid_count_columns = [col for col in count_columns if col in df.columns]
        valid_median_columns = [col for col in median_columns if col in df.columns]
        
        # Compute yearly count 
        if valid_count_columns:
            df[f'{year}-YearlyCount'] = df[valid_count_columns].sum(axis=1)
        
        # Compute yearly median 
        if valid_median_columns:
            df[f'{year}-YearlyMedian'] = df[valid_median_columns].mean(axis=1)
    
    return df

# Years list 
years = list(range(2000, 2023))  

yearly_df = calculate_yearly_values(df, years)
yearly_df.head()

,suburb,Mar 2000-Count,Mar 2000-Median,Jun 2000-Count,Jun 2000-Median,Sep 2000-Count,Sep 2000-Median,Dec 2000-Count,Dec 2000-Median,Mar 2001-Count,...,2018-YearlyCount,2018-YearlyMedian,2019-YearlyCount,2019-YearlyMedian,2020-YearlyCount,2020-YearlyMedian,2021-YearlyCount,2021-YearlyMedian,2022-YearlyCount,2022-YearlyMedian
2,Albert Park-Middle Park-West St Kilda,1143.0,260.0,1134,260,1177.0,270.0,1178.0,275.0,1208.0,...,3649,562.5,3495,568.0,3342,570.0,3695,498.75,3435,510.0
3,Armadale,733.0,200.0,737,200,738.0,205.0,739.0,210.0,718.0,...,3667,487.5,3067,481.25,2977,498.75,3289,433.75,3348,447.5
4,Carlton North,864.0,260.0,814,260,799.0,265.0,736.0,270.0,718.0,...,2020,566.25,1986,579.25,2164,588.75,2391,577.5,2209,593.75
5,Carlton-Parkville,1339.0,260.0,1304,260,1300.0,260.0,1320.0,260.0,1273.0,...,17683,417.0,17946,431.25,19712,399.75,19582,335.0,25659,329.75
6,CBD-St Kilda Rd,2132.0,320.0,2264,320,2358.0,320.0,2361.0,320.0,2591.0,...,40842,493.75,43018,505.0,46880,471.25,69064,362.25,70548,406.0


In [6]:
# Merge horizontally, with the yearly records starting from the 187th column of the yearly_df
past_rent_cleaned = pd.concat([yearly_df['suburb'],yearly_df.iloc[:,187:]],axis=1)

# Remove group total records
past_rent_cleaned = past_rent_cleaned[past_rent_cleaned['suburb'] != 'Group Total']

past_rent_cleaned.head()

,suburb,2000-YearlyCount,2000-YearlyMedian,2001-YearlyCount,2001-YearlyMedian,2002-YearlyCount,2002-YearlyMedian,2003-YearlyCount,2003-YearlyMedian,2004-YearlyCount,...,2018-YearlyCount,2018-YearlyMedian,2019-YearlyCount,2019-YearlyMedian,2020-YearlyCount,2020-YearlyMedian,2021-YearlyCount,2021-YearlyMedian,2022-YearlyCount,2022-YearlyMedian
2,Albert Park-Middle Park-West St Kilda,4632.0,266.25,4990.0,281.25,4097,300.0,6844,295.0,4963,...,3649,562.5,3495,568.0,3342,570.0,3695,498.75,3435,510.0
3,Armadale,2947.0,203.75,2904.0,222.5,2330,231.0,3729,236.0,2839,...,3667,487.5,3067,481.25,2977,498.75,3289,433.75,3348,447.5
4,Carlton North,3213.0,263.75,2737.0,276.25,1917,290.0,3426,290.0,2580,...,2020,566.25,1986,579.25,2164,588.75,2391,577.5,2209,593.75
5,Carlton-Parkville,5263.0,260.0,5144.0,263.75,3834,271.666667,6630,281.2,6569,...,17683,417.0,17946,431.25,19712,399.75,19582,335.0,25659,329.75
6,CBD-St Kilda Rd,9115.0,320.0,11012.0,320.0,10127,320.0,19318,308.0,16879,...,40842,493.75,43018,505.0,46880,471.25,69064,362.25,70548,406.0


###  Matched with postcode

In [7]:
def get_postcode(suburb, api_key):
    '''  
        This function is used to find the postcode by suburb name by using API from
        Aupost.
    '''

    url = 'https://digitalapi.auspost.com.au/postcode/search.json?'
    
    # Define key and parameters
    headers = {'auth-key': api_key}
    params = {'q': suburb, "state": "VIC"}
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    # One suburb can have multiple matched postcode, only consider first one
    if 'localities' in data and 'locality' in data['localities']:
        if isinstance(data['localities']['locality'],list):
            return data['localities']['locality'][0]['postcode']
        else:
            return data['localities']['locality']['postcode']
    
    return None

In [8]:
# Make call of the function to find postcode
api_key = 'ac59205f-785a-42fd-81a5-813887d322fb' # YOU OWN API
past_rent_cleaned['postcode'] = past_rent_cleaned.apply(lambda row: get_postcode(row['suburb'], api_key), axis=1)

In [9]:
rows_with_none_postcode = past_rent_cleaned[past_rent_cleaned['postcode'].isna()]
print("Number of undefined suburb: ", len((rows_with_none_postcode.suburb)))

Number of undefined suburb:  59


In [11]:
# Manually assign postcodes
manual_postcode_mapping = {
    'Albert Park-Middle Park-West St Kilda': '3004',
    'Carlton-Parkville': '3053',
    'CBD-St Kilda Rd': '3004',
    'Collingwood-Abbotsford':'3067',
    'Fitzroy North-Clifton Hill':'3068',
    'Flemington-Kensington':'3031',
    'North Melbourne-West Melbourne':'3051',
    'Prahran-Windsor':'3181',
    'Richmond-Burnley':'3121',
    'Bulleen-Templestowe-Doncaster':'3105',
    'Burwood-Ashburton':'3125',
    'Camberwell-Glen Iris':'3146',
    'Canterbury-Surrey Hills-Mont Albert':'3127',
    'Chadstone-Oakleigh':'3166',
    'Doncaster East-Donvale':'3109',
    'Glen Waverley-Mulgrave':'3150',
    'Nunawading-Mitcham':'3132',
    'Vermont-Forest Hill-Burwood East':'3133',
    'Aspendale-Chelsea-Carrum':'3195',
    'Hampton-Beaumaris':'3193',
    'Mentone-Parkdale-Mordialloc':'3195',
    'Murrumbeena-Hughesdale':'3163',
    'Keilor East-Avondale Heights':'3034',
    'Newport-Spotswood':'3015',
    'St Albans-Deer Park':'3023',
    'Werribee-Hoppers Crossing':'3029',
    'Yarraville-Seddon':'3013',
    'Broadmeadows-Roxburgh Park':'3064',
    'Coburg-Pascoe Vale South':'3044',
    'Gladstone Park-Tullamarine':'3034',
    'Moonee Ponds-Ascot Vale':'3039',
    'Oak Park-Glenroy-Fawkner':'3046',
    'Pascoe Vale-Coburg North':'3044',
    'Bundoora-Greensborough-Hurstbridge':'3099',
    'Eltham-Research-Montmorency':'3094',
    'Fairfield-Alphington':'3078',
    'Mill Park-Epping':'3082',
    'Thomastown-Lalor':'3075',
    'Croydon-Lilydale':'3414',
    'Wantirna-Scoresby':'3179',
    'Yarra Ranges':'3160',
    'Dandenong North-Endeavour Hills':'3802',
    'Narre Warren-Hampton Park':'3976',
    'Dromana-Portsea':'3936',
    'Hastings-Flinders':'3915',
    'Mt Eliza-Mornington-Mt Martha':'3934',
    'Seaford-Carrum Downs':'3201',
    'Belmont-Grovedale':'3216',
    'Geelong-Newcombe':'3219',
    'Herne Hill-Geelong West':'3218',
    'Mount Clear-Buninyong':'3350',
    'Sebastopol-Delacombe':'3356',
    'Wendouree-Alfredton':'3350',
    'Flora Hill-Bendigo East':'3550',
    'Golden Square-Kangaroo Flat':'3555',
    'Moe-Newborough':'3825',
    'Ocean Grove-Barwon Heads':'3227',
    'Sale-Maffra':'3860',
    'Wanagaratta':'3677'
}

past_rent_cleaned['postcode'] = past_rent_cleaned.apply(
    lambda row: manual_postcode_mapping.get(row['suburb'], row['postcode']),
    axis=1
)

print("Number of undefined suburb after assignment: ", past_rent_cleaned['postcode'].isna().sum())

past_rent_cleaned.to_csv("../data/curated/past_rent_cleaned_postcode.csv")


Number of undefined suburb after assignment:  0


## SA2 Area

Australian postcode can be download from [here](https://github.com/matthewproctor/australianpostcodes).

In [16]:
# Load the dataset
zones = gpd.read_file( "../data/raw/external/SA2_2021_ShapeFile/SA2_2021_AUST_GDA2020.shp")
rent_df = pd.read_csv("../data/curated/rent_cleaned.csv")

postcode = pd.read_csv("../data/raw/external/australian_postcodes.csv")
postcode_vic = postcode.loc[postcode['state']=='VIC']
postcode_vic = postcode_vic[['lat','long','locality','postcode']]

zones = zones.loc[zones['GCC_NAME21'].isin(['Rest of Vic.','Greater Melbourne'])]
zones['geometry'] = zones['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [17]:
geometry = [Point(xy) for xy in zip(postcode_vic['long'], postcode_vic['lat'])]
postcode_gdf = gpd.GeoDataFrame(postcode_vic, geometry=geometry, crs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs:4326')

In [18]:
# Spatial join
sa2_with_postcodes = gpd.sjoin(zones, postcode_gdf, how='left',  predicate='intersects')

# Drop rows where one sa2 area matched with one postcode multiple times
sa2_with_postcodes.drop_duplicates(['SA2_CODE21','postcode'],inplace=True)

# Check unmatched rows
print("Number of unmatched SA2 area: ",(sa2_with_postcodes['postcode'].isna()).sum())

# Split sa2_with_postcodes df into 2 dfs
unmatched_sa2 = sa2_with_postcodes[sa2_with_postcodes['postcode'].isna()] #168
matched_sa2 = sa2_with_postcodes[~sa2_with_postcodes['postcode'].isna()] #828

Number of unmatched SA2 area:  168


For records with unmatched postcodes, use the AusPostcode API to retrieve the  postcode.

In [19]:
def get_postcode(suburb, api_key):
    '''  
        This function is used to find the postcode by suburb name by using API from
        Aupost.
    '''

    url = 'https://digitalapi.auspost.com.au/postcode/search.json?'
    
    # Define key and parameters
    headers = {'auth-key': api_key}
    params = {'q': suburb, "state": "VIC"}
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    # One suburb can have multiple matched postcode, only consider first one
    if 'localities' in data and 'locality' in data['localities']:
        if isinstance(data['localities']['locality'],list):
            return data['localities']['locality'][0]['postcode']
        else:
            return data['localities']['locality']['postcode']
    
    return None

In [66]:
print("Number of unmathced sa2 before: ", unmatched_sa2['postcode'].isna().sum())

api_key = 'ac59205f-785a-42fd-81a5-813887d322fb' # YOUR OWN API
unmatched_sa2.loc[:,'postcode'] = unmatched_sa2.apply(lambda row: get_postcode(row['SA2_NAME21'], api_key), axis=1)

print("Number of unmathced sa2 after: ", unmatched_sa2['postcode'].isna().sum())

Number of unmathced sa2 before:  97
Number of unmathced sa2 after:  97


Since all postcodes in the past rent DataFrame can be found in the SA2 DataFrame, any unmatched postcodes will not impact future modeling steps. Therefore, we will stop attempting to match any remaining unmatched postcodes.

In [21]:
# Merge two dfs
new_sa2_postcode = pd.concat([unmatched_sa2,matched_sa2],axis=0)
sa2_postcode_list = list(sa2_with_postcodes['postcode'].unique()) # Extract the postcodes

# Get the postcodes from past rent file
past_rent = pd.read_csv("../data/curated/past_rent_cleaned_postcode.csv")
rent_postcode_list = list(past_rent['postcode'].unique())

# Check is all postcode in past rent df is in sa2
print(set(rent_postcode_list)-set(sa2_postcode_list))

# Save the result 
new_sa2_postcode.to_file('../data/curated/sa2_with_postcodes.shp')

set()


/var/folders/9w/m54rf3zj3gn16gj55q9v_2j80000gn/T/ipykernel_13123/1542953785.py:13: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  new_sa2_postcode.to_file('../data/curated/sa2_with_postcodes.shp')
/Users/user/anaconda3/lib/python3.11/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'index_right' to 'index_righ'
  ogr_write(


## Population and income

In [57]:
# Read the file
sa2_with_postcodes = gpd.read_file('../data/curated/sa2_with_postcodes.shp')
sa2_with_postcodes['SA2_CODE21'] = sa2_with_postcodes['SA2_CODE21'].astype('float')

income = pd.read_csv("../data/curated/income_with_affluence.csv")
population = pd.read_csv("../data/curated/population_forecast.csv")

In [58]:
# Merge income with population
pop_income = pd.merge(population, income, how='left',right_on='SA2', left_on='SA2 code')

# Merge them with SA2 with postcode
pop_income_postcode = pd.merge(pop_income, sa2_with_postcodes, how='left', left_on='SA2',right_on='SA2_CODE21')

# Drop unnecessary columns
pop_income_postcode.drop(['GCCSA name', 'SA2_CODE21',
       'SA2_NAME21', 'CHG_FLAG21', 'CHG_LBL21', 'SA3_CODE21', 'SA3_NAME21',
       'SA4_CODE21', 'SA4_NAME21', 'GCC_CODE21', 'GCC_NAME21', 'STE_CODE21',
       'STE_NAME21', 'AUS_CODE21', 'AUS_NAME21', 'AREASQKM21', 'LOCI_URI21',
       'index_righ', 'lat', 'long', 'locality'], axis=1, inplace=True)
# Rename
pop_income_postcode.rename(columns={'growth_rate_x':'population_growth_rate','growth_rate_y':'income_growth_rate'}, inplace=True)

# Save
pop_income_postcode.to_csv("../data/curated/pop_income_postcode.csv")
pop_income_postcode.head()

,SA2 code,SA2 name,2001,2002,2003,2004,2005,2006,2007,2008,...,2016-17,2017-18,2018-19,2019-20,2020-21,average_income,income_growth_rate,affluence_index,postcode,geometry
0,201011001,Alfredton,5756,6092,6293,6480,6648,6761,7034,7272,...,50596.0,52448.0,53932.0,55204.0,58036.0,54043.2,0.147047,37830.284114,3350.0,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5..."
1,201011002,Ballarat,11497,11708,12015,12189,12269,12356,12408,12480,...,50093.0,51736.0,53688.0,53784.0,55998.0,53059.8,0.117881,37141.895364,3354.0,"POLYGON ((143.81896 -37.55582, 143.81644 -37.5..."
2,201011005,Buninyong,5320,5399,5557,5620,5857,6037,6131,6252,...,48877.0,51034.0,52377.0,54308.0,56408.0,52600.8,0.154081,36820.606224,3357.0,"POLYGON ((143.84171 -37.61596, 143.84176 -37.6..."
3,201011006,Delacombe,4154,4225,4371,4465,4704,5041,5206,5349,...,46176.0,47759.0,49909.0,51915.0,52792.0,49710.2,0.143278,34797.182983,3352.0,"POLYGON ((143.7505 -37.59119, 143.75044 -37.59..."
4,201011006,Delacombe,4154,4225,4371,4465,4704,5041,5206,5349,...,46176.0,47759.0,49909.0,51915.0,52792.0,49710.2,0.143278,34797.182983,3358.0,"POLYGON ((143.7505 -37.59119, 143.75044 -37.59..."


## Merge Everything

In [59]:
# Read file
pop_income_postcode = pd.read_csv("../data/curated/pop_income_postcode.csv")
past_rent = pd.read_csv("../data/curated/past_rent_cleaned_postcode.csv")

In [63]:
whole_df = pd.merge(past_rent, pop_income_postcode, how='left', on='postcode')
whole_df.head() # 209 records, some suburb with multiple postcode

,Unnamed: 0_x,suburb,2000-YearlyCount,2000-YearlyMedian,2001-YearlyCount,2001-YearlyMedian,2002-YearlyCount,2002-YearlyMedian,2003-YearlyCount,2003-YearlyMedian,...,SA2 NAME,2016-17,2017-18,2018-19,2019-20,2020-21,average_income,income_growth_rate,affluence_index,geometry
0,2,Albert Park-Middle Park-West St Kilda,4632.0,266.25,4990.0,281.25,4097,300.0,6844,295.0,...,East Melbourne,69469.0,71759.0,75755.0,77804.0,81648.0,75287.0,0.175316,52700.952595,POLYGON ((144.9739585748236 -37.81330241814959...
1,2,Albert Park-Middle Park-West St Kilda,4632.0,266.25,4990.0,281.25,4097,300.0,6844,295.0,...,South Yarra - West,64865.0,68512.0,70451.0,69578.0,72567.0,69194.6,0.118739,48436.255622,POLYGON ((144.97455041239857 -37.8346653683224...
2,2,Albert Park-Middle Park-West St Kilda,4632.0,266.25,4990.0,281.25,4097,300.0,6844,295.0,...,Albert Park,65352.0,66627.0,67518.0,68933.0,73107.0,68307.4,0.118665,47815.215600,POLYGON ((144.96766973330057 -37.8373669078686...
3,2,Albert Park-Middle Park-West St Kilda,4632.0,266.25,4990.0,281.25,4097,300.0,6844,295.0,...,St Kilda - Central,53464.0,56940.0,59644.0,60523.0,64644.0,59043.0,0.209113,41330.162734,POLYGON ((144.9822465081365 -37.86154000901546...
4,3,Armadale,2947.0,203.75,2904.0,222.50,2330,231.0,3729,236.0,...,Armadale,63296.0,66098.0,69563.0,72012.0,73321.0,68858.0,0.158383,48200.647515,POLYGON ((145.01167433388778 -37.8535692509816...


In [64]:
# All past rental records has postcode 
whole_df['postcode'].isna().sum()

0

In [65]:
whole_df.to_csv("../data/curated/past_rent_income_population.csv")